In [10]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('airbnb').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/22 19:54:33 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.159 instead (on interface en0)
26/01/22 19:54:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/22 19:54:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/22 19:54:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
airbnb_search = spark.read.options(header=True, inferSchema=True, sep='~')\
    .csv('data/input/airbnb_search_details.csv')
    
airbnb_search.show(truncate=False)

+--------+------+-------------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------+--------+-------------------+------------+----+----------------------+------------------+----------+-------------------+-----------------+--------------------+-------+--------+----+
|id      |price |property_type|room_type      |amenities                                                                                                                                                                                                                 

In [21]:
spark._jvm.org.apache.log4j.LogManager.getLogger("org.apache.spark.sql.execution.window.WindowExec").setLevel(spark._jvm.org.apache.log4j.Level.ERROR)

result = airbnb_search.select('city', 'amenities')\
    .withColumn('amenity_count', F.size(F.split('amenities', ',')))\
    .groupBy('city').agg(F.sum('amenity_count').alias('amenity_count'))\
    .withColumn('max_amenity_count', F.max('amenity_count').over(Window.partitionBy().orderBy(F.col('amenity_count').desc())))\
    .filter(F.col('amenity_count') == F.col('max_amenity_count'))\
    .select('city')

result.show(truncate=False)

+----+
|city|
+----+
|NYC |
+----+

